### This can be the final functionized version, where running the tests is easy - just change 'n' and 'bound'
### It is fast at sieving, but is not factoring properly - can you take a look at anything that looks off? 
#### Need to only check after B smooth numbers have already been generated

In [234]:
import math
import numpy as np
import sympy
import time 

In [235]:
def calc_time(start, end):
    seconds = end - start
    mins = seconds / 60
    print('Time in seconds: ', str(seconds), ', Time in minutes: ', str(mins))
    return seconds, mins

In [236]:
# Final optimized version 
def get_primes_less_thanB(bound):                      # eratosthenes way
    check_until = int(bound**0.5)
    unmarked = np.array([x for x in range(2, bound)])
    q = 2
    i = 0
    while True:
        u = unmarked[i]
        if u> check_until:               # only check numbers up to sqrt(bound)
            break
        u_idx = np.where(unmarked == u)[0][0]
        max_idx = bound//u
        listof_multiples = [u*i for i in range(q, max_idx+1)]
        u_multiples = np.array(listof_multiples)
        unmarked = np.setdiff1d(unmarked, u_multiples)   # numpy set diff without changing order of elements in list
        q = u
        i +=1
    print("Number of elements less than bound = ", bound, " is: ", len(unmarked))
    return (unmarked)

In [237]:
# Correct optimized version
def check_Bsmooth(primes, num):
    powers_of_primes= []
    num_remaining= num                         
    for prime in primes: 
        i = 0                                   # i = power of prime
        if (num_remaining%prime == 0):
            while (num_remaining%prime ==0):   #keep dividing for each power of the prime
                quotient = num_remaining/prime
                num_remaining = quotient
                i +=1
        powers_of_primes.append(i)
        if num_remaining == 1:     
            diff = len(primes)-len(powers_of_primes)
            if diff>0:
                zeros = [0 for k in range(diff)]
                return powers_of_primes + zeros
            return powers_of_primes
    #print("Number is not B smooth")
    return []

In [238]:
# Correct optimized version
def check_Bsmooth(primes, num):
    powers_of_primes= []
    num_remaining= num                         
    for prime in primes: 
        i = 0                                   # i = power of prime
        if (num_remaining%prime == 0):
            while (num_remaining%prime ==0):   #keep dividing for each power of the prime
                quotient = num_remaining/prime
                num_remaining = quotient
                i +=1
        powers_of_primes.append(i)
        if num_remaining == 1:     
            diff = len(primes)-len(powers_of_primes)
            if diff>0:
                zeros = [0 for k in range(diff)]
                return powers_of_primes + zeros
            return powers_of_primes
    #print("Number is not B smooth")
    return []

In [239]:
def a_congruent_b(a, b, n):
    if a%n==(b%n):
        return True
    return False

In [240]:
def generate_BSsquares(primes, n, multiple):
    exponents_list = []             # Add other fancy ways of finding candidates for xsquare
    xsquares = []           # the x
    x_vals = []
    residues = []           # the y
    iters = 0
    while(True):
       # if iters>100000000:
       #     break
        x = int(math.sqrt(n*multiple)) +1
        xsquare = x**2
        residue = xsquare%n
        exp_vec = check_Bsmooth(primes, residue)
        if exp_vec != []:
            residues.append(residue)   # x^2
            xsquares.append(xsquare)
            exponents_list.append(exp_vec)
            x_vals.append(x)
            print("Found B-smooth square! with exponents: ", exp_vec, " and quadratic residue: ", residue)
        if len(residues) > len(primes)+10 :
            print("Breaking, len of residues: ", len(residues), " len of primes: ", len(primes))
            break
        multiple += 1
        iters += 1
    return [exponents_list, residues, iters, xsquares, x_vals]

In [241]:
def drop_zero_rows(A): 
    sums = np.sum(A, axis = 1)
    evens = []
    
    for i in range(0, len(sums)):
        if(sums[i]==0):
            evens.append(i)
    
    
    return [A[~np.all(A == 0, axis=1)], evens]

In [242]:
def get_product(v, num_list):  # just multiply all RELEVANT (depends on v) numbers in a list together
    product= 1       
    for i in range(len(v)):
        if v[i] ==0:
            continue
        product = product * num_list[i]
#         product = product%n    # not reducing mod n yet
    return product

In [243]:
def get_root_modN(square, n): # xsq is the number left after multiplying all RELEVANT x squares found, check next func
    result = int(math.sqrt(square))%n
    return result

In [244]:
# Choose a subset of columns (equations) to be 1 - those with 0 in pivot or 1 but the rowsum adds to 0mod2
def getA_solution(L):
    #L = reduced_matrix
    col_num = len(L[0])
    row_num = len(L)
    v = np.ones(col_num, dtype = 'int')
    for i in reversed(range(row_num)):
        for j in range(col_num):
            if L[i][j] ==1:
                if (L[i].sum()%2) !=0:                     # the exponents for this prime doesn't add up to zero
                    v[j] = 0                  # can't use this equation
                    break     # no need to  check cells above, as reduced echelon form
    return(v)


In [245]:
#aug_matrix is the row reduced matrix mod 2. 
# It is not augmented, I just kept the name like that because I used some old code from a previous implementation
# v_combs is set up to be [0...01], which means that the last free variable only will be set to 1
# indicies_of_free_vars includes all indices of columns which don't have a pivot

def set_v_comb_array(aug_matrix):
   
    # first set v to have all 0s for pivot nums, 1 for free nums
    col_num = len(aug_matrix[0])
    row_num = len(aug_matrix)
    v = np.ones(col_num, dtype = 'int')
    
    for i in range(0, row_num):
        check_main = 1
        for j in range(i, col_num):
            if(aug_matrix[i][j]==1 and check_main ==1):
                v[j]=0
                break
    
    # generate v_combs
    count_free = np.sum(v); # gives us the count of the free variables.
    v_combs = np.zeros(count_free, dtype = 'int')
    v_combs[count_free-1] = 1;
    indices_of_free_vars = np.zeros(count_free, dtype= 'int')
    
    # generate indices_of_free_vars
    k=0;
    for i in range(0, col_num):
        if(v[i]==1):
            indices_of_free_vars[k] = i;
            k+=1;
    
    return [v_combs, indices_of_free_vars]

In [246]:
#input - the row reduced matrix, the combination array, the indices array from the function above, rows and columns numbers
#output - the vector used to calculate x, y. It has 0s for the variables that are not used, 1s for the variables that are used.

def calc_v(L, v_combs, v_indices, rows, cols):
    # create r, such that the only 1s are the free variables specified by v_combs
    # the rest of the free variables are 2s, and the non-free ones are 0s
    
    r = np.zeros(cols, dtype ='int')
    for i in range(0, len(v_combs)):
        if(v_combs[i]==1):
            r[v_indices[i]] = 1;
        else:
            r[v_indices[i]] =2;  
    
    # I am using the code from a previous implementation to calculate the rest of the values in r. 
    
    for i in reversed(range(0, rows)):
    #for j in range(0, col_num):
        sum =0
        for j in range(0, cols):
            sum = sum + r[j]*L[i][j];
        zer =(np.argwhere(r == 0).flatten())
        sum = sum%2;
        if(len(zer)!=0):
            index = zer[len(zer)-1]
    
            if(sum==0):
                r[index] = 2
            else:
                r[index] = 1

    print("final r vector is ", r%2)
    return r%2

In [265]:
# I use this function to do the combinations of free variables.
# It works on the following principle:
# Each index of the array from which we want to generate combinations is either represented or not in the combination
# Hence, the indices of the array can be represented in an array of 0s and 1s
# Example:
# Array: [2, 3, 4, 5, 6]
# comb_arr: [0, 0, 1, 0, 0]
# combination: [4]

# comb_arr: [0, 0, 1, 1, 0]
# combination: [4, 5]

# comb_arr: [1, 0, 1, 1, 0]
# combination: [2, 4, 5]

# Hence, by starting from comb_arr = [0,0,0,...,0, 1] and incrementing the array similarly to a binary number, we get 
# an exhaustive list of all possible combinations of k elements.

def binary_add_arr(comb_arr):
    #for i in reverse(range(0, len(comb_arr))):
    i = len(comb_arr) -1
    while(comb_arr[i]==1):
        comb_arr[i] = 0;
        i-=1;
    if(i>=0):
        comb_arr[i] = 1;
    else:
        return -1
    return comb_arr        

In [248]:
# This function overflows for very large nums, and I am still trying to understand why.
# Instead of squaring the y, I add all of the powers of its primes, and then divide the sums by 2
# and multiply the prime**power_of_the_prime, like what we did before.
# Since this method uses only addition, multiplication, and division, I though it would work better than squaring,
# but it still gives wrong answers. I think it might be because the np library uses a 32-bit int instead of a big int?
# Or we have a float creeping in in some of the operations which ruins everything?

def generate_y(pows, r, primes_arr):
    primes = len(pows[0])
    final_pows = np.zeros(primes)
    
    for i in range(0, len(r)):
        container = 0
        for j in range(0, primes):
            final_pows[j]+=r[i]*pows[i][j]
    final_pows//=2;
    final_num = 1;
    #final_pows = int(final_pows)
    
    print("final_pows: ",final_pows)
    
    for i in range(0, len(final_pows)):
        if(primes_arr[i]!=0 and final_pows[i]!=0):
            final_num*=primes_arr[i]**(final_pows[i]);
    return final_num

k = generate_y([[1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3, 3]], [1, 0, 1], [2, 3, 5, 7])
print(k)

final_pows:  [2. 2. 2. 2.]
44100.0


In [249]:
# That's a function I found here https://riptutorial.com/python/example/8751/computing-large-integer-roots
# It calculates accuretely roots of large nums, but I have not invested time to understand how and why it works
# so we can do our own implementation.

def nth_root(x, n):
    # Start with some reasonable bounds around the nth root.
    upper_bound = 1
    while upper_bound ** n <= x:
        upper_bound *= 2
    lower_bound = upper_bound // 2
    # Keep searching for a better result as long as the bounds make sense.
    while lower_bound < upper_bound:
        mid = (lower_bound + upper_bound) // 2
        mid_nth = mid ** n
        if lower_bound < mid and mid_nth < x:
            lower_bound = mid
        elif upper_bound > mid and mid_nth > x:
            upper_bound = mid
        else:
            # Found perfect nth root.
            return mid
    return mid + 1

print(nth_root(7062487582582867647322134510298657004160000, 2))

2657534116917949869600


#### Choose the bound value and a N of your liking (8-15 digits)

In [266]:
#n = 95775679 - works
n = 6817540046645387
#n = 6817540046645387
k = math.log(n)
k = k*math.log(k)
k = math.exp(math.sqrt(k))
k = math.pow(k, math.sqrt(2)/4)
bound = int(k)

primes = np.array(get_primes_less_thanB(bound))

Number of elements less than bound =  57  is:  16


In [252]:
# A test on a small prime!
# n = 10710336959293
    
# n = 1147
#n = 1147
multiple = 1
start = time.time()

results = generate_BSsquares(primes, n, multiple)

end = time.time()
calc_time(start, end)
print("iterations, ", results[2])
print("B smooth numbers found: ", results[4], "\ntotal: ", len(results[1]))

Found B-smooth square! with exponents:  [1, 10, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]  and quadratic residue:  5689371150
Found B-smooth square! with exponents:  [0, 0, 1, 0, 0, 2, 1, 2, 0, 1, 0, 0, 0, 0, 0, 1]  and quadratic residue:  7970520805
Found B-smooth square! with exponents:  [5, 0, 0, 1, 0, 1, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0]  and quadratic residue:  1633477664
Found B-smooth square! with exponents:  [0, 5, 0, 3, 1, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0]  and quadratic residue:  21337594047
Found B-smooth square! with exponents:  [1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]  and quadratic residue:  1041238
Found B-smooth square! with exponents:  [0, 6, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0]  and quadratic residue:  559754631
Found B-smooth square! with exponents:  [0, 0, 2, 2, 1, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]  and quadratic residue:  6542611075
Found B-smooth square! with exponents:  [4, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0]  and quadratic residue:  15558150608
F

### The following uses (results, n) produced from last run cell 

In [253]:
from sympy import *          # Start using np arrays

A = np.array(results[0])      # results = [exponents_list, residues, iters, xsquares]
store_pows = A.copy()
A = (np.transpose(A))%2
print(A)

[[1 0 1 0 1 0 0 0 1 0 1 0 0 1 1 1 0 1 0 0 1 1 0 1 1 1 0]
 [0 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0]
 [0 0 1 1 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 1 1 0 1]
 [0 0 0 1 1 0 1 1 1 0 1 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1]
 [0 1 0 1 0 1 1 0 0 1 1 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0]
 [0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0]
 [0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1]
 [0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 1]
 [1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0]]


In [254]:
col_num = len(A[0])
row_num = len(A)
print("Old num rows and num columns: ", row_num, col_num)
L = Matrix(A)

L = np.array(L.rref()[0])%2
print(L)# reduced row echelon form
L2  = drop_zero_rows(L)[0]        # get rid of primes we don't have to worry about in general
L2 = L2 %2
    

Old num rows and num columns:  16 27
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1/2 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1/2 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1/2 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/2 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 3/2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1/2 0]]


In [255]:
# This cell is still a stub, I haven't worked on it a lot, but the idea is that 0-rows actually mean
# that we have a b-smooth number which is already a square. Hence, we can combine those numbers
# and look for a solution among them too. 

# This will help us when working with no so large numbers, since they tend to have smaller matrices 
# and more zero rows.


# check for a case where many of the rows have disappeared because they are already even
even_arr = drop_zero_rows(L)[1]
print(even_arr)

[]


In [267]:
print("Reduced row echelon form AND mod 2 is: ")
print(L2)
print("New num rows, num columns: ", len(L2), len(L2[0]))

Reduced row echelon form AND mod 2 is: 
[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 1/2 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1/2 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1/2 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1/2 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 3/2 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1/2 0]]
New num rows, num columns:  16 27


In [257]:
col_num = len(L2[0])
row_num = len(L2)

# initialize the combination array, and the corresponding free variables indices array

v_combs, v_indices = set_v_comb_array(L2)

#calculate the inital v vector
v = calc_v(L2, v_combs, v_indices, row_num, col_num)     # tells you which equations we can choose


final r vector is  [0 1 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1]


In [264]:
# results is of form: [exponents_list, residues, iters, xsquares, x_vals (x before squaring)]
residues = results[1]    # x%n, RHS
xsquares = results[3]

#print(residues)
#print(xsquares)
ysq = get_product(v, residues) 
xsq = get_product(v, xsquares)

print("final x squared mod n: ", xsq%n, " final y squared mod n: ", ysq%n) # just a check - they have to be equal
print(ysq)

[5689371150, 7970520805, 1633477664, 21337594047, 1041238, 559754631, 6542611075, 15558150608, 4295617920, 46198980700, 13041567000, 31882083220, 5437496875, 21527871834, 6533910656, 4164952, 2239018524, 40881852550, 26170444300, 8131770549, 38262664286, 9418395350, 71734687245, 838369280, 14701298976, 11224121888, 62232602432]
final x squared mod n:  3310072439289510  final y squared mod n:  3310072439289510
11116958414832437956949500413229220511894268565568949474866694297138932449931545160258849600000000000


In [259]:
x_vals = results[4]
#print(x_vals, primes)
x = get_product(v, x_vals)
y = nth_root(ysq, 2)
print('y is ', y)


# here we check if the x and y are good, and until they become good we change the combination of free variables
while ((x-y)%n==0 or (x+y)%n==0 or(y-x)%n==0):
    v_combs = binary_add_arr(v_combs)
    v = calc_v(L2, v_combs, v_indices, row_num, col_num)
    x = get_product(v, x_vals)
    ysq = get_product(v, residues) 
    y = nth_root(ysq, 2)
    
print("\n")
print("Found x not congruent y mod n! A Factor!!")  # THIS ISN'T TRUE! GCD(x-y)doesn't return the right value :"(
print(x, y)
print('current combs are: ',v_combs)

[5422550159, 14892555617, 16348333935, 16764028031, 17525907944, 17796123323, 20279241867, 25328318161, 26719903561, 26878137327, 27190338271, 29785111234, 29811879437, 31276533554, 32696667870, 35051815888, 35592246646, 36560368388, 40558483734, 41683630766, 42157615146, 43740447139, 44677666851, 48500973911, 49045001805, 50417522985, 50656636322] [ 2  3  5  7 11 13 17 19 23 29 31 37 41 43 47 53]
y is  37946574498079355288904032530380365606400
final r vector is  [1 1 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0]


Found x not congruent y mod n! A Factor!!
60599471676432879734326550311821361450101657591890632775761692862764647163909659840473167974103305930280 105436987887706835595207564356767653609127576201429
current combs are:  [0 0 0 0 0 0 0 0 0 1 0]


In [260]:
diff = abs(x-y)

print("x-y is: ", x-y)
gcd = math.gcd(diff, n)
print("GCD found is: ", gcd)
print("other factor of n is: ", int(n/gcd))


x-y is:  60599471676432879734326550311821361450101657591890632670324704975057811568702095483705514364975729728851
GCD found is:  1
other factor of n is:  6817540046645387


### THE END